# A Dense Retrieval using DPR

Install the needed packages - dataasets and faiss-cpu

In [ ]:
%%capture
%pip install datasets faiss-cpu

In [ ]:
import torch

from datasets import load_dataset
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer

## Data Preprocessing and Indexing

Load the **context** DPR models from huggingface hub.  
They will be used to encode the context, the document data.

In [ ]:
torch.set_grad_enabled(False)

ctx_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
ctx_tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/492 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizer'.


Load a dataset (Crime and Punishment)

In [ ]:
ds = load_dataset('crime_and_punish', split='train[:100]'); ds

README.md:   0%|          | 0.00/5.14k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/786k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/21969 [00:00<?, ? examples/s]

Dataset({
    features: ['line'],
    num_rows: 100
})

In [ ]:
ds[:20]

{'line': ['CRIME AND PUNISHMENT\r\n',
  '\r\n',
  '\r\n',
  '\r\n',
  '\r\n',
  'PART I\r\n',
  '\r\n',
  '\r\n',
  '\r\n',
  'CHAPTER I\r\n',
  '\r\n',
  'On an exceptionally hot evening early in July a young man came out of\r\n',
  'the garret in which he lodged in S. Place and walked slowly, as though\r\n',
  'in hesitation, towards K. bridge.\r\n',
  '\r\n',
  'He had successfully avoided meeting his landlady on the staircase. His\r\n',
  'garret was under the roof of a high, five-storied house and was more\r\n',
  'like a cupboard than a room. The landlady who provided him with garret,\r\n',
  'dinners, and attendance, lived on the floor below, and every time\r\n',
  'he went out he was obliged to pass her kitchen, the door of which\r\n']}

Prepare the data to be indexed: convert each line to an *embedding vector*

In [ ]:
ds_with_embeddings = ds.map(
    lambda example:
     {'embeddings': ctx_encoder(**ctx_tokenizer(example["line"], return_tensors="pt"))[0][0].numpy()})

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

... and index them

In [ ]:
ds_with_embeddings.add_faiss_index(column='embeddings')

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['line', 'embeddings'],
    num_rows: 100
})

In [ ]:
ds_with_embeddings

Dataset({
    features: ['line', 'embeddings'],
    num_rows: 100
})

## Retrieval

Load the **question** DPR models from huggingface hub.  
They will be used to encode the query.

In [ ]:
q_encoder = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
q_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-question_encoder-single-nq-base")

config.json:   0%|          | 0.00/493 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Tokenize and encode a query:

In [ ]:
question = "Is it serious ?"
question_embedding = q_encoder(**q_tokenizer(question, return_tensors="pt"))[0][0].numpy()

and perform retrieval

In [ ]:
scores, retrieved_examples = ds_with_embeddings.get_nearest_examples('embeddings', question_embedding, k=5)
retrieved_examples["line"]

['_that_ serious? It is not serious at all. It’s simply a fantasy to amuse\r\n',
 'and complaints, and to rack his brains for excuses, to prevaricate, to\r\n',
 'CRIME AND PUNISHMENT\r\n',
 'for him. But to be stopped on the stairs, to be forced to listen to her\r\n',
 'trivial, irrelevant gossip, to pestering demands for payment, threats\r\n']

In [ ]:
scores

array([ 94.92376 ,  95.6583  ,  98.723434, 103.86665 , 106.68605 ],
      dtype=float32)

Although we created the index inside the huggingface dataset, it can still be accessed through the `.faiss_index` property.  
We can use it for [more operations and methods](https://github.com/facebookresearch/faiss/wiki/Special-operations-on-indexes), such as [Range Search](https://github.com/facebookresearch/faiss/wiki/Special-operations-on-indexes#range-search)

In [ ]:
faiss_index = ds_with_embeddings.get_index('embeddings').faiss_index
faiss_index

<faiss.swigfaiss_avx2.IndexFlat; proxy of <Swig Object of type 'faiss::IndexFlat *' at 0x7b91dcc41e90> >

In [ ]:
# access the index with .faiss_index
faiss_index = ds_with_embeddings.get_index('embeddings').faiss_index

question = "murder"
question_embedding = q_encoder(**q_tokenizer(question, return_tensors="pt"))[0][0].numpy()

# use it for more complicated methods - e.g., range_search
limits, distances, indices = faiss_index.range_search(x=question_embedding.reshape(1, -1), thresh=0.5)

limits, distances, indices

(array([0, 0], dtype=uint64), array([], dtype=float32), array([], dtype=int64))

In [ ]:
distances, indices = faiss_index.search(x=question_embedding.reshape(1, -1), k=3)
distances, indices


(array([[94.92376 , 95.6583  , 98.723434]], dtype=float32),
 array([[50, 34,  0]]))

## Disk operations
How to save and load the index:

In [ ]:
# to save:
ds_index_path = 'my_index.faiss'
ds_index_name = 'embeddings'

ds_with_embeddings.save_faiss_index(ds_index_name, ds_index_path)

In [ ]:
# to load:
ds = load_dataset('crime_and_punish', split='train[:100]')
ds.load_faiss_index(ds_index_name, ds_index_path)